In [1]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Set model variables
OPENAI_BASE_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION")

# Initialize LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Demos"

## Set up Agent: Memory Sentinel


In [2]:

from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables import RunnablePassthrough





In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt_initial),
        MessagesPlaceholder(variable_name='messages'),
        (
            "system",
            "Remember, only respond with TRUE or FALSE. Do not provide any other info"
        ),
    ]
)


llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    streaming = True,
    temperature=0.0
)

sentinel_runnable = {"messages"  : RunnablePassthrough()} | prompt | llm

## Set up Agent: Memory Manager



In [4]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool
from enum import Enum
from typing import Optional


class Category(str, Enum):
    Activities_liking = "Likings"
    Activities_disliking = "Dislikings"
    Career_goals = 'Career_goals'
    Family_attribute = "Family_attribute"
    
class Action(str, Enum):
    Create = "Create"
    Update = "Update"
    Delete = "Delete"
    
class AddKnowledge(BaseModel):
    knowledge: str = Field(
        ...,
        description="Condensed bit of knowledge to be saved for future reference in the format: [person(s) this is relevant to] [fact to store] (e.g. User doesn't drin alcohol; user has a girlfriend; etc)",
    )
    knowledge_old: Optional[str] = Field(
        ..., description='Category that this knowledge belongs to'
    )
    category: Category = Field(
        ..., description="Category that this knowledge belongs to"
    )
    action: Action = Field(
        ...,
        description="Whether this knowledge is adding a new record, updating a record, or deleting a record",
    )
    

def modify_knowledge(
    knowledge: str,
    category: str,
    action: str,
    knowledge_old: str = "",
) -> dict:
    print("Modifying knowledge: ", knowledge, knowledge_old, category, action)
    return "Modified knowledge"

tool_modify_knowledge = StructuredTool.from_function(
    func = modify_knowledge, # the function to run when the tool is called
    name = "Knowledge_Modifier",
    description="Add, update, or delete a bit of knowledge", # Used to tell the model how/when/why to use the tool.
    args_schema=AddKnowledge #The input arguments’ schema.
)




In [5]:
from langgraph.prebuilt import ToolExecutor


agent_tools = [tool_modify_knowledge]
tool_executor = ToolExecutor(agent_tools)



In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.utils.function_calling import convert_to_openai_function

system_prompt_initial =  """
You are a supervisor managing a team of knowledge eperts.

Your team's job is to create a perfect knowledge base about a person's plans, likings, habits to assist in highly customized and effective day planning.

The knowledge base should ultimately consist of many discrete pieces of information that add up to a rich persona (e.g. I like reading; I do not eat fastfood; i love working; I live in Prague; I have a girlfriend called Lisa).

Every time you receive a message, you will evaluate if it has any information worth recording in the knowledge base.

A message may contain multiple pieces of information that should be saved separately.

You are only interested in the following categories of information:

1. The person's goals in career  (to find a new Machine Learning posisiton)
2. Outdoor activities the person is interested in.
3. Person's hated activities (e.g. doesn't like smoking, drinking in the night clubs etc)
4. Attributes about the person that may impact weekly planning (e.g. lives in Prague; has a girlfriend/boyfriend/wife/husband; Has 2000$ dollar sallary; likes big lunches; etc.)
5. Dreams, personal goals of a person (e.g attending a list of amazng restraunts, Visiting Italy, reading N quantity of books etc)
6. Person's likings and dislikings.

When you receive a message, you perform a sequence of steps consisting of:

1. Analyze the most recent Human message for information. You will see multiple messages for context, but we are only looking for new information in the most recent message.
2. Compare this to the knowledge you already have.
3. Determine if this is new knowledge, an update to old knowledge that now needs to change, or should result in deleting information that is not correct. It's possible that an activity/fact you previously wrote as a dislike might now be a like, or that a person changed his/her mind and now want to try some new experience - those examples would require an update.

Here are the existing bits of information that we have about the person.

```
{memories}
```

Call the right tools to save the information, then respond with DONE. If you identiy multiple pieces of information, call everything at once. You only have one chance to call tools.

I will tip you $20 if you are perfect, and I will fine you $40 if you miss any important information or change any incorrect information.

Take a deep breath, think step by step, and then analyze the following message:
"""


prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt_initial),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    #model="gpt-4-0125-preview",
    streaming=True,
    temperature=0.0,
)

#create the tools to bind the model:

tools = [convert_to_openai_function(t) for t in agent_tools]


knowledge_master_runnable = prompt | llm.bind_tools(tools)

## Set The Graph

In [6]:
from typing import TypedDict, Sequence
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    #List of previous messages in the conversation
    messages: Sequence[BaseMessage]
    #The Long - Term Memories to remember 
    memories: Sequence[str]
    #Whether The info is relevant
    contains_information: str
    
    
    
    
    

`ToolMessage` Message for passing the result of executing a tool back to a model.

Pass in content as positional arg.


In [8]:
import json
from langchain_core.messages import ToolMessage
from langgraph.prebuilt import ToolInvocation

def call_sentinel(state):
    messages = state['messages']
    #last_message = messages[-1]
    response = sentinel_runnable.invoke(messages)
    return {"contain_infromation": "TRUE" in response.content and "yes" or "no"}


#another way
"""
if "TRUE" in response.content:
    return {"contains_information": "yes"}
else:
    return {"contains_information": "no"}
    
..."""


def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    else:
        return "continue"
    
# Define the function that calls the knowledge master

def call_knowledge_master(state):
    messages = state['messages']
    memories = state['memories']
    response = knowledge_master_runnable.invoke(
        {'messages' : messages, "memories" : memories}
    )
    return {'messages' : messages + [response]}

def call_tool(state):
    messages = state['messages']
    #we know the last message involves at least one tool
    last_message = messages[-1]
    #loop through all tool calls and append the message to our message log
    for tool_call in last_message.additional_kwargs['tool_calls']:
        action = ToolInvocation(
            tool=tool_call['function']['name'],
            tool_input = json.loads(tool_call['function']['arguments']),
            id = tool_call['id']
        )
        response = tool_executor.invoke(action)
        
        function_message= ToolMessage(
            content = str(response), name=action.tool, tool_call_id = tool_call['id']
        )
        
        messages.append(function_message)
    return {"messages" : messages}


In [10]:
from langgraph.graph import StateGraph, END

# Initialize a new graph
graph = StateGraph(AgentState)

# Define the two "Nodes"" we will cycle between
graph.add_node("sentinel", call_sentinel)
graph.add_node("knowledge_master", call_knowledge_master)
graph.add_node("action", call_tool)

# Define all our Edges

# Set the Starting Edge
graph.set_entry_point("sentinel")

# We now add Conditional Edges
graph.add_conditional_edges('sentinel',
                            lambda x: x['contains_information'],
                            {   'yes' : "knowledge_master",
                                "no" : END, 
                            }
)

graph.add_conditional_edges('knowledge_master',
                            should_continue,
                            {
                                "continue" : "action", 
                                "end" : END
                            }
)

# We now add Normal Edges that should always be called after another
graph.add_edge('action', END )

app = graph.compile()

In [11]:
from langchain_core.messages import HumanMessage

message = "My name is Mike. I love coding and build startups"

inputs = {
    "messages" : [HumanMessage(content=message)],
}



In [15]:
for output in app.with_config({'run_name': "Memory"}).stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

InvalidUpdateError: Invalid state update, expected dict with one or more of ['messages', 'memories', 'contains_information'], got {'contain_infromation': 'yes'}